In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
from code_map import Inputs, old_markets, old_meters
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


407.0


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/Jobb/Volte/master-kode/code_map/markets.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffr_df["FFR-Flex Price [EUR/MW]"][(pd.Timestamp(year = year, month =10, day = 30, hour = 0, tz = "Europe/Oslo") < ffr_df["Time(Local)"]) &
/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/Jobb/Volte/master-kode/code_map/markets.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffr_df["FFR-Profil Price [EUR/MW]"][(pd.Timestamp(year = year, month = 9, day = 3, hour = 0, tz = "Europe/Oslo") < ffr_df["Time(Local)"]) &
/Users/sandermeland/Documents/Dokum

In [2]:
i_v = Inputs.version_3_variables
H = pd.date_range(start=pd.Timestamp(year= i_v.year, month= i_v.start_month, day = i_v.start_day, hour = i_v.start_hour), 
                              end= pd.Timestamp(year = i_v.year, month = i_v.end_month, day = i_v.end_day, hour = i_v.end_hour), freq="H", tz = "Europe/Oslo")

power_meter_dict = old_meters.power_meters

# Define the sets
L = list(power_meter_dict.values())  # List of PowerMeter objects
M = old_markets.all_market_list  # List of ReserveMarket objects

In [3]:
len(L)

1455

In [4]:
F_h_l = np.array([[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H]) # set of flex volumes for meters

#L_s = np.array([[meter.sleep_time for meter in L]] * len(H)) # set of sleep times for meters
R_h_l = np.array([[meter.response_time for meter in L]] * len(H)) # set of response times for meters

P_h_m = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of prices for markets
Vp_h_m = np.array([[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of volumes for markets

Vm_m = [market.min_volume for market in M] # set of min values for markets
R_m = [market.response_time for market in M] # set of response times for markets
#M_s = np.array([[market.sleep_time for market in M]] * len(H)) # set of sleep times for markets"""

In [5]:
I = {}
for l in range(len(L)):
    for m in range(len(M)): 
        for h in range(len(H)):
            I[h,l,m] = F_h_l[h,l] * P_h_m[h,m] 

In [6]:
compatible_list = []
for h, hour in enumerate(H):
    hour_list = []
    for l, asset in enumerate(L):
        asset_list = []
        for m, market in enumerate(M):
            if asset.direction == "up":
                if market.direction == "up":
                    asset_list.append(m)
                else:
                    continue
            elif asset.direction == "down":
                if market.direction == "down":
                    asset_list.append(m)
                else:
                    continue
            elif asset.direction == "both":
                asset_list.append(m)
        hour_list.append(asset_list)
    compatible_list.append(hour_list)

                
    
    

In [7]:
# Create a new model
test_model = Model("AssetToMarket")

# Create decision variables
x = {}
d = {}
for h in range(len(H)):
    for l in range(len(L)):
        for m in range(len(M)):
            # asset i is connected to market j at hour h
            x[h, l, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"x_{h}_{l}_{m}")

            d[h,l,m] = 1 if m in compatible_list[h][l] else 0
            
            # adding the constraint
            test_model.addConstr(x[h,l,m] <= d[h,l,m])
            

y = {}
for h in range(len(H)):
    for m in range(len(M)):
        # market j is active at hour h
        y[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"y_{h}_{m}")


# Set objective
test_model.setObjective(
    # maxmize income 
    sum(x[h, l, m] *I[h, l, m] for l in range(len(L)) for m in range(len(M)) for h in range(len(H))), GRB.MAXIMIZE)

# Add constraints
for h in range(len(H)):
    for l in range(len(L)):
        # Each asset can only be connected to one market at a time
        test_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")

    for m in range(len(M)):
        # Connect the binary variables by using big M
        test_model.addConstr((sum(x[h, l, m] for l in range(len(L)))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
        
        # Min volume constraint
        test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes

        # Weighted average constraint for response time
        total_response_time = sum(x[h, l, m] * R_h_l[h,l] for l in range(len(L)))
        total_assigned_assets = sum(x[h, l, m] for l in range(len(L)))
        test_model.addConstr(total_response_time <= R_m[m] * total_assigned_assets, f"avg_response_time_for_hour_{h}_market_{m}")

        # Max volume constraint
        test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")

# Solve the model
test_model.optimize()
    
if test_model.status == GRB.Status.INFEASIBLE:
    test_model.computeIIS()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8927448 rows, 8316672 columns and 48945843 nonzeros
Model fingerprint: 0xd3c0a613
Variable types: 0 continuous, 8316672 integer (8316672 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e-07, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 7666858 rows and 188410 columns (presolve time = 11s) ...
Presolve removed 8314224 rows and 5844036 columns (presolve time = 15s) ...
Presolve removed 8402075 rows and 5849802 columns (presolve time = 20s) ...
Presolve removed 8402075 rows and 5849802 columns (presolve time = 27s) ...
Pres

### Test the old version in the same file because of the random values being initiated in the beginning of the file.

In [ ]:
old_test_model = Model("AssetToMarket")

# Create decision variables
x_old = {}
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        for m, market in enumerate(M):
            # asset i is connected to market j at hour h
            x_old[h, l, m] = old_test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"x_{h}_{l}_{m}")
            
            # If asset direction is "up"
            if load.direction == "up" and market.direction == "down":
                old_test_model.addConstr(x_old[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If asset direction is "down"
            elif load.direction == "down" and market.direction == "up":
                old_test_model.addConstr(x_old[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If market direction is "both"
            elif market.direction == "both" and load.direction != "both":
                old_test_model.addConstr(x_old[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If asset direction is "any", we don't need a constraint as it can connect to any market.

y_old = {}
for h, hour in enumerate(H):
    for m, market in enumerate(M):
        # market j is active at hour h
        y_old[h, m] = old_test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"y_{h}_{m}")


# Set objective
old_test_model.setObjective(
    # maxmize income 
    sum(x_old[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] * market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for l, load in enumerate(L) for m, market in enumerate(M) for h, hour in enumerate(H)),
    GRB.MAXIMIZE
)

# Add constraints
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        # Each asset can only be connected to one market at a time
        old_test_model.addConstr(sum(x_old[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")

for h, hour in enumerate(H):
    for m, market in enumerate(M):
        # Connect the binary variables by using big M
        old_test_model.addConstr((sum(x_old[h, l, m] for l in range(len(L)))) <= len(L) * y_old[h, m], f"asset_connection_for_hour_{h}_market_{m}")
        
        # Min volume constraint
        old_test_model.addConstr(sum(x_old[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L)) >= market.min_volume * y_old[h, m], f"min_volume_for_hour_{h}_market_{m}")

        # Weighted average constraint for response time
        total_response_time = sum(x_old[h, l, m] * load.response_time for l, load in enumerate(L))
        total_assigned_assets = sum(x_old[h, l, m] for l in range(len(L)))
        old_test_model.addConstr(total_response_time <= market.response_time * total_assigned_assets, f"avg_response_time_for_hour_{h}_market_{m}")

        # Max volume constraint
        old_test_model.addConstr(sum(x_old[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L)) <=  market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]  * y_old[h,m], f"max_volume_for_hour_{h}_market_{m}")

# Solve the model
old_test_model.optimize()
    
if old_test_model.status == GRB.Status.INFEASIBLE:
    old_test_model.computeIIS()

In [14]:
def get_market_count_dict(x):
    data = []

    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    data.append([hour, load.meter_id, market.name])

    df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
    #print(df)


    market_count_dict = {}
    for hour in H:
        hour_df = df.loc[(df["Hour"] == hour)]
        # get the number of assets in each market
        market_count = hour_df.groupby(["Market", "Hour"]).agg("count").reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
        market_count_dict[hour] = market_count
    return market_count_dict

In [15]:
market_count_dict = get_market_count_dict(x)


In [ ]:
market_count_dict[H[247]]

In [ ]:
old_model_market_count_dict = get_market_count_dict(x_old)

In [ ]:
for h, hour in enumerate(H):
    if market_count_dict[hour].equals(old_model_market_count_dict[hour]):
        #print(f"Hour: {hour} is equal")
        continue
    else:
        print(f"Hour: {hour} is not matching")
        print(f"New model: {market_count_dict[hour]}")
        print(f"Old model: {old_model_market_count_dict[hour]}")
    

In [10]:
def test_solution_validity(x, y, L, M, H):
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert sum(x[h, l, m].X for m in range(len(M))) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x[h, l, m].X == 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x[h, l, m].X == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                elif market.direction == "both" and load.direction != "both":
                    assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"

        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert sum(x[h, l, m].X for l in range(len(L))) <= len(L) * y[h, m].X, f"More than allowed assets connected at hour {h} to market {m}"

            # Min volume constraint
            total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))
            assert total_flex_volume >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"

            # Weighted average constraint for response time
            total_response_time = sum(x[h, l, m].X * load.response_time for l, load in enumerate(L))
            total_assigned_assets = sum(x[h, l, m].X for l in range(len(L)))
            assert total_response_time <= market.response_time * total_assigned_assets, f"Average response time constraint violated at hour {h} for market {m}"

            # Max volume constraint
            total_max_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * y[h,m].X, f"Maximum volume constraint violated at hour {h} for market {m}"





In [17]:
test_solution_validity(x, y, L, M, H)


AssertionError: Minimum volume constraint violated at hour 65 for market 6

In [16]:
market_count_dict[H[65]]

,Market,Hour,Asset Count
0,RK_down,2023-06-16 17:00:00+02:00,1284
1,aFRR up,2023-06-16 17:00:00+02:00,105


In [20]:
M[6].name

'aFRR up'

In [19]:
sum(F_h_l[65, :])

11.396366500000008

In [ ]:
test_solution_validity(x_old, y_old, L, M, H)